In [24]:
import pandas as pd # For Data Storage
import requests # For website connection
from bs4 import BeautifulSoup # For HTML parsing
import time # For sleep 
import re # Regular expressions for removing non-ascii terms
from itertools import groupby # For removing duplicates from lists
import math # Need ceiling expression
from nltk.corpus import stopwords # For filtering out words like 'is', 'the', 'of'
stop_words = set(stopwords.words("english")) #initialize stopwords    
import matplotlib.pyplot as plt

# Initialize Empty Lists (For Dictionary)

In [8]:
#Initialize Empty State Set
state = []
job_title = []

#Initialize Empty Skills List
r = []
python = []
java = []
cpp = []
ruby = []
perl = []
matlab = []
javascript = []
scala = []
excel = []
vba = []
tableau = []
d3 = []
hadoop = []
mapreduce = []
spark = []
pig = []
hive = []
shark = []
oozie = []
zookeeper = []
flume = []
mahout = []
sql = [] 
mysql = []
oracle = []
postgresql = []
nosql =[]
sybase = []
rethinkdb = []
memcacheddb = []
couchdb = []
hbase = []
cassandra = []
mongodb = []
db2 = []

skills = ['r',
'python',
'java',
'c++',
'ruby',
'perl',
'matlab',
'javascript',
'scala', 
'excel',
'vba',
'tableau',
'd3',
'hadoop',
'mapreduce',
'spark',
'pig',
'hive',
'shark',
'oozie',
'zookeeper',
'flume',
'mahout',
'sql', 
'mysql',
'oracle',
'postgresql',
'nosql',
'sybase',
'rethinkdb',
'memcacheddb',
'couchdb',
'hbase',
'cassandra',
'mongodb',
'db2'
]

# Initialize Dictionary

In [9]:
dictionary = {'job_title':job_title,
     'state':state,
     'r':r,
     'python':python,
     'java':java,
     'c++':cpp,
     'ruby':ruby,
     'perl':perl,
     'matlab':matlab,
     'javascript':javascript,
     'scala':scala,
     'excel':excel,
     'vba':vba,
     'tableau':tableau,
     'd3':d3,
     'hadoop':hadoop,
     'mapreduce':mapreduce,
     'spark':spark,
     'pig':pig,
     'hive':hive,
     'shark':shark,
     'oozie':oozie,
     'zookeeper':zookeeper,
     'flume':flume,
     'mahout':mahout,
     'sql':sql, 
     'mysql':mysql,
     'oracle':oracle,
     'postgresql':postgresql,
     'nosql':nosql,
     'sybase':sybase,
     'rethinkdb':rethinkdb,
     'memcacheddb':memcacheddb,
     'couchdb':couchdb,
     'hbase':hbase,
     'cassandra':cassandra,
     'mongodb':mongodb,
     'db2':db2}

# Define Func: Add A Row of Data To Dictionary (For a Job Description)

In [10]:
def addValsToDict(url_state,url_job_title):
    #Append job_title
    dictionary['job_title'].append(url_job_title)
    
    #Append state
    dictionary['state'].append(url_state)

    #Check if the each skill is in the wrangled text
    for skill in skills:
        if (skill in text_final) == True:
            dictionary[skill].append(1)
        else:
            dictionary[skill].append(0)

# Define States List

In [11]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

## For Each State:

- ## Get the urls that lists up to 250 job openings

In [12]:
count = 0

# Initialize Empty URL Dictionary
final_state_url_dict = {} 

for url_state in states:
    count += 1
    print("Number of states left to scrape: "+str(50-count))
    
    url = 'https://www.monster.com/jobs/search/?q=data-scientist&where='+url_state+'&stpage=1'
    response = requests.get(url) #Get the html code
    soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 

    time.sleep(1) #pause code for .01 sec
    
    #Get the total number of listed jobs 
    totalJobOpenings = float(re.sub('[^0-9]','',soup.html.body.find_all('h2')[0].text.strip()))
    
    #Get the total number of pages (up to 10)
    totalPages = min(math.ceil(totalJobOpenings/25),10)
    
    #Store the Final URL in a dictionary each state
    final_state_url_dict[url_state] = 'https://www.monster.com/jobs/search/?q=data-scientist&where='+url_state+'&stpage=1&page='+str(totalPages)

- ## Get the list of job post URLs -> For each URL: check if each skill exists and store data in dictionary

In [13]:
count = 0
for state in final_state_url_dict:
    count += 1
    #Access the job listing page for a state
    response = requests.get(final_state_url_dict[state]) #Get the html code
    soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 

    links = []
    job_titles = {}
    # Access every URL within the page and store only urls for job openings
    for link in soup.find_all('a'):
        if link.get('href')[0:21] == 'https://job-openings.': ## a job opening always begins with 'https://job-openings.monster.com'
            links.append(link.get('href'))
            job_titles[link.get('href')] = link.contents[0].strip('\n').strip('\r')

    time.sleep(1) #pause code for .01 sec
    
    print("Number of states left to scrape: "+str(50-count))
    
    # Access each job posting and store it's html code into a beautiful soup object
    for url in links:
        response = requests.get(url) #Get the html code
        soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 
        
        #Get and clean the text from the job posting
        text = soup.body.find_all('script')[1]

        # Convert to string -> Convert to lower case 
        text_wrangled = str(text).lower()
        # replace (all non ascii characters,'.' ,'+' , and '3') with ','
        text_wrangled = re.sub("[^a-zA-Z.+3]",',',text_wrangled)
        # split string into a list separated by ','
        text_wrangled = text_wrangled.split(',')
        # remove all consecutive duplicates 
        text_wrangled = groupby(text_wrangled)
        text_wrangled = [x[0] for x in text_wrangled]
        # Remove the stopwords and empty spaces
        text_final = [x for x in text_wrangled if not x in stop_words and not x =='']
        #Add the values to the Dictionary
        addValsToDict(state,job_titles[url])

Number of states left to scrape: 49
Number of states left to scrape: 48
Number of states left to scrape: 47
Number of states left to scrape: 46
Number of states left to scrape: 45
Number of states left to scrape: 44
Number of states left to scrape: 43
Number of states left to scrape: 42
Number of states left to scrape: 41
Number of states left to scrape: 40
Number of states left to scrape: 39
Number of states left to scrape: 38
Number of states left to scrape: 37
Number of states left to scrape: 36


IndexError: list index out of range

# Make the DataFrame

In [14]:
#for x in dictionary:
#    print(len(dictionary[str(x)]))
data = pd.DataFrame(dictionary)

In [30]:
data.head(5)

job_title state  r  python  java  c++  ruby  \
0                         Special Agent    AL  0       0     0    0     0   
1  Software Architect - Enterprise Java    AL  0       0     1    0     0   
2                        Data Scientist    AL  1       1     1    0     1   
3                Data Scientist, Junior    AL  1       1     0    0     0   
4                        Data Scientist    AL  1       1     1    0     0   

   perl  matlab  javascript ...   postgresql  nosql  sybase  rethinkdb  \
0     0       0           0 ...            0      0       0          0   
1     0       0           0 ...            0      0       0          0   
2     0       1           0 ...            0      0       0          0   
3     0       0           1 ...            0      0       0          0   
4     0       0           1 ...            0      1       0          0   

   memcacheddb  couchdb  hbase  cassandra  mongodb  db2  
0            0        0      0          0        0    0  
1            0        0      0          0        0    0  
2            0        0      0          0        0    0  
3            0        0      0          0        0    0  
4            0        0      0          0        0    0  

[5 rows x 38 columns]

In [54]:
response = requests.get(url,proxies={123.195.152.139})

SyntaxError: invalid syntax (<ipython-input-54-5ae4792dce2e>, line 1)

'https://job-openings.monster.com/analytics-translator-indianapolis-in-us-eli-lilly-company/50f6cbdd-5f93-4327-ad07-6c5484a1c57c'

In [55]:
url

'https://job-openings.monster.com/analytics-translator-indianapolis-in-us-eli-lilly-company/50f6cbdd-5f93-4327-ad07-6c5484a1c57c'

In [56]:
s = requests.Session()
s.get("http://www.baidu.com", proxies={'http': 'http://10.11.4.254:3128'})


ProxyError: HTTPConnectionPool(host='10.11.4.254', port=3128): Max retries exceeded with url: http://www.baidu.com/ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a219000f0>: Failed to establish a new connection: [Errno 60] Operation timed out',)))

In [57]:
proxies = {'http': 'http://10.11.4.254:3128'}
s = requests.session()
s.proxies.update(proxies)
s.get("http://www.example.com") 

ProxyError: HTTPConnectionPool(host='10.11.4.254', port=3128): Max retries exceeded with url: http://www.example.com/ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a21420be0>: Failed to establish a new connection: [Errno 60] Operation timed out',)))

In [61]:
proxies = {"https": "173.192.21.89:25"}

requests.get("https://dataandthings.science.blog/", proxies=proxies)

ProxyError: HTTPSConnectionPool(host='dataandthings.science.blog', port=443): Max retries exceeded with url: / (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 403 Forbidden',)))